In [2]:
!python -V

Python 3.9.12


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import numpy as np

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn import preprocessing, svm

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

In [8]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("new-experiment")

2023/04/12 23:29:15 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 23:29:15 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='file:///c:/Users/Shashank/OneDrive/Documents/MLOPS-Zoomcamp/mlops-zoomcamp-main/mlops-zoomcamp-main/02-experiment-tracking/mlruns/1', creation_time=1681337027733, experiment_id='1', last_update_time=1681337027733, lifecycle_stage='active', name='new-experiment', tags={}>

In [9]:
filename = "../../../../../../Downloads/student_scores.csv"

In [10]:
df = pd.read_csv(filename)

    
    


In [11]:
y = df['Scores'].values.reshape(-1, 1)
X = df['Hours'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [12]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [13]:
#Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

5.234206765787757

In [14]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump( lr, f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Shashank")

    mlflow.log_param("train-data-path", "../../../../../../Downloads/student_scores.csv")
    

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="lin_reg.bin", artifact_path="models_pickle")

In [17]:
!pip install -r requirements.txt

  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached xgboost-1.7.5-py3-none-win_amd64.whl (70.9 MB)
  Using cached fastparquet-2023.2.0-cp39-cp39-win_amd64.whl (619 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached pandas-2.0.0-cp39-cp39-win_amd64.whl (11.3 MB)
  Using cached cramjam-2.6.2-cp39-none-win_amd64.whl (1.2 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


In [18]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [24]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[23:31:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:49.41987                          
[1]	validation-rmse:45.27325                          
[2]	validation-rmse:41.46824                          
[3]	validation-rmse:37.97964                          
[4]	validation-rmse:34.70850                          
[5]	validation-rmse:31.70939                          
[6]	validation-rmse:28.97144                          
[7]	validation-rmse:26.44811                          
[8]	validation-rmse:24.17767                          
[9]	validation-rmse:22.04890                          
[10]	validation-rmse:20.15890                         
[11]	validation-rmse:18.41755                         
[12]	validation-rmse:16.85138                         
[13]	validation-rmse:15.35680                         

In [25]:
mlflow.xgboost.autolog(disable=True)

In [26]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_test, label=y_test)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[23:36:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:48.51208
[1]	validation-rmse:43.61868
[2]	validation-rmse:39.03077
[3]	validation-rmse:34.90015
[4]	validation-rmse:31.17145
[5]	validation-rmse:27.82429
[6]	validation-rmse:24.83668
[7]	validation-rmse:22.15804
[8]	validation-rmse:19.78339
[9]	validation-rmse:17.77126
[10]	validation-rmse:15.79489
[11]	validation-rmse:14.23960
[12]	validation-rmse:12.71258
[13]	validation-rmse:11.54992
[14]	validation-rmse:10.45159
[15]	validation-rmse:9.50770
[16]	validation-rmse:8.75429
[17]	validation-rmse:8.13955
[18]	validation-rmse:7.68821
[19]	validation-rmse:7.37420
[20]	validation-rmse:7.17676
[21]	validation-rmse:7.10553
[22]	validation-rmse:7.05170
[23]	validation-rmse:7.08759
[24]	validation-rmse:7.16608
[25]	validation-rmse:7.24626
[26]	valid

2023/04/12 23:36:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Shashank\AppData\Local\Temp\tmp6l1gx9jb\model, flavor: xgboost), fall back to return ['xgboost==1.7.5']. Set logging level to DEBUG to see the full traceback.
c:\Users\Shashank\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [27]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", filename)

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2023/04/12 23:37:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Shashank\anaconda3\lib\site-packages\mlflow\utils\autologging_utils\safety.py:532: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."
c:\Users\Shashank\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
2023/04/12 23:37:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Shashank\anaconda3\lib\site-packages\mlflow\utils\autologging_utils\safety.py:532: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."
c:\Users\Shashank\anaconda3\lib\si